In [12]:
import pandas as pd
import numpy as np
import torch
from argparse import Namespace
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Add the parent directory of 'ml' to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import warnings
warnings.filterwarnings('ignore')


from ml.utils.data_utils import prepare_dataset
from ml.models.lstm import LSTM
from ml.models.multi_step_lstm import MultiStepLSTM

In [2]:
# ─────────────────────────────────────────────────────────────
# 0) CONFIG
args = Namespace(
    data_path='../dataset/full_dataset.csv',
    targets=['rnti_count', 'rb_down', 'rb_up', 'down', 'up'],
    num_lags=10,
    forecast_steps=6,
    test_size=0.2,
    ignore_cols=None,          # keep if you don't want to drop any
    identifier='District',
    nan_constant=0,
    x_scaler='minmax',
    y_scaler='minmax',
    outlier_detection=True,
    # Model hyperparams (must match what you used when training and saving the weights)
    hidden_size=128,
    num_layers=1,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    seed=0
)

torch.manual_seed(args.seed)
np.random.seed(args.seed)

In [3]:
# Where are the 5 targets located inside X's feature dimension?
# If your input feature vector is [rnti_count, rb_down, rb_up, down, up, <maybe_other_feature>],
# the indices for the 5 targets are [0,1,2,3,4].
TARGET_POS_IN_X = [0, 1, 2, 3, 4]   # <-- CHANGE IF NEEDED

# Filenames of saved models
BASE_MODEL_PATH   = "base_lstm_t1.pt"
MULTISTEP_PATH    = "multi_step_lstm.pt"

In [4]:
# ─────────────────────────────────────────────────────────────
# 1) DATA
X_train, y_train, X_test, y_test, x_scaler, y_scaler, id_train, id_test = prepare_dataset(args)
# y_* is shape [N, 6, 5] (6 steps, 5 targets). For base LSTM evaluation we’ll use only t+1 labels.
y_train_t1 = y_train[:, 0, :]      # [N, 5]
y_test_t1  = y_test[:, 0, :]       # [N, 5]

N, L, D = X_test.shape
output_dim = len(args.targets)

In [6]:
# Base paper LSTM (predicts t+1 only)
base_model = LSTM(
    input_dim=D,
    lstm_hidden_size=args.hidden_size,      # must match the hidden size used when training
    num_lstm_layers=2,                      # <-- checkpoint has 2 LSTM layers
    lstm_dropout=0.0,                       # use what you trained with; dropout doesn't affect state_dict keys
    layer_units=[128, 64],                  # <-- checkpoint has two hidden Linear layers before the output
    num_outputs=len(args.targets),          # 5
    matrix_rep=True,                        # your X is [batch, seq_len, feat]
    exogenous_dim=0
).to(args.device)

state = torch.load(BASE_MODEL_PATH, map_location=args.device)
base_model.load_state_dict(state, strict=True)  # should load cleanly now
base_model.eval()

LSTM(
  (lstm): LSTM(6, 128, num_layers=2, batch_first=True)
  (MLP_layers): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=5, bias=True)
  )
)

In [7]:
# Multi-step LSTM (direct 6-step prediction)
multi_model = MultiStepLSTM(
    input_size=D,
    hidden_size=args.hidden_size,
    num_layers=args.num_layers,
    output_size=output_dim,
    forecast_steps=args.forecast_steps
).to(args.device)

multi_model.load_state_dict(torch.load(MULTISTEP_PATH, map_location=args.device))
multi_model.eval()

MultiStepLSTM(
  (lstm): LSTM(6, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=30, bias=True)
)

In [8]:
# ─────────────────────────────────────────────────────────────
# 3) METRICS

def safe_mape(y_true, y_pred, eps=1e-8):
    denom = np.clip(np.abs(y_true), eps, None)
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0

def eval_metrics(y_true, y_pred, name):
    mse  = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    nrmse = rmse / (y_true.max() - y_true.min() + 1e-8)
    mape = safe_mape(y_true, y_pred)
    print(f"\n{name}:")
    print(f"  MSE   : {mse:.6f}")
    print(f"  RMSE  : {rmse:.6f}")
    print(f"  MAE   : {mae:.6f}")
    print(f"  R²    : {r2:.6f}")
    print(f"  NRMSE : {nrmse:.6f}")
    print(f"  MAPE% : {mape:.3f}")
    return dict(MSE=mse, RMSE=rmse, MAE=mae, R2=r2, NRMSE=nrmse, MAPE=mape)

In [ ]:
# ─────────────────────────────────────────────────────────────
# 4) STRATEGY A: t+1 vs t+1
with torch.no_grad():
    xb = torch.tensor(X_test, dtype=torch.float32).to(args.device)
    base_t1 = base_model(xb, device=args.device)         # [N, 5]
    multi_all = multi_model(xb)                           # [N, 6, 5]
base_t1  = base_t1.cpu().numpy()
multi_t1 = multi_all[:, 0, :].cpu().numpy()

print("\n===== Strategy A: Compare t+1 vs t+1 =====")
for i, var in enumerate(args.targets):
    # Ground truth t+1
    y_true_t1 = y_test_t1[:, i]
    # Base LSTM t+1
    eval_metrics(y_true_t1, base_t1[:, i], name=f"Base t+1 — {var}")
    # Multi-step LSTM t+1
    eval_metrics(y_true_t1, multi_t1[:, i], name=f"Multi t+1 — {var}")


===== Strategy A: Compare t+1 vs t+1 =====

Base t+1 — rnti_count:
  MSE   : 0.008045
  RMSE  : 0.089696
  MAE   : 0.066063
  R²    : 0.485215
  NRMSE : 0.125351
  MAPE% : 30.246

Multi t+1 — rnti_count:
  MSE   : 0.006707
  RMSE  : 0.081893
  MAE   : 0.062210
  R²    : 0.570880
  NRMSE : 0.114447
  MAPE% : 33.774

Base t+1 — rb_down:
  MSE   : 0.008326
  RMSE  : 0.091249
  MAE   : 0.043579
  R²    : 0.559291
  NRMSE : 0.091397
  MAPE% : 56.740

Multi t+1 — rb_down:
  MSE   : 0.008512
  RMSE  : 0.092262
  MAE   : 0.047042
  R²    : 0.549452
  NRMSE : 0.092412
  MAPE% : 59.624

Base t+1 — rb_up:
  MSE   : 0.012164
  RMSE  : 0.110291
  MAE   : 0.052190
  R²    : 0.610206
  NRMSE : 0.110291
  MAPE% : 149655.780

Multi t+1 — rb_up:
  MSE   : 0.013288
  RMSE  : 0.115274
  MAE   : 0.056397
  R²    : 0.574183
  NRMSE : 0.115274
  MAPE% : 60585.607

Base t+1 — down:
  MSE   : 0.007340
  RMSE  : 0.085676
  MAE   : 0.050102
  R²    : 0.480701
  NRMSE : 0.085995
  MAPE% : 44.474

Multi t+1 — dow

In [10]:
# ─────────────────────────────────────────────────────────────
# 5) STRATEGY B: Roll base LSTM to t+6 (recursive) and compare step-by-step

def roll_base_lstm_to_horizon(base_model, X_init, steps, target_pos_in_x, device="cpu"):
    """
    base_model: LSTM that predicts next-step (t+1) targets, returns [N, 5]
    X_init:     [N, L, D] initial window
    steps:      number of recursive steps (e.g., 6)
    target_pos_in_x: positions in input feature vector where the 5 targets live
    """
    base_model.eval()
    x_win = torch.tensor(X_init, dtype=torch.float32, device=device)  # [N, L, D]
    out_steps = []
    with torch.no_grad():
        for t in range(steps):
            y_next = base_model(x_win, device=device)  # [N, 5]
            out_steps.append(y_next.unsqueeze(1))      # [N,1,5]

            # Build updated last row
            last_row = x_win[:, -1, :].clone()         # [N, D]
            for k, pos in enumerate(target_pos_in_x):
                last_row[:, pos] = y_next[:, k]

            # Shift the window: drop oldest, append updated last_row
            x_win = torch.cat([x_win[:, 1:, :], last_row.unsqueeze(1)], dim=1)

    return torch.cat(out_steps, dim=1).cpu().numpy()     # [N, steps, 5]

base_rolled = roll_base_lstm_to_horizon(
    base_model, X_test, steps=args.forecast_steps, target_pos_in_x=TARGET_POS_IN_X, device=args.device
)  # [N, 6, 5]

# Multi-step predictions already computed above (multi_all)
multi_preds = multi_all.cpu().numpy()  # [N, 6, 5]

print("\n===== Strategy B: Step-by-step comparison (t+1…t+6) =====")
for step in range(args.forecast_steps):
    print(f"\n-- Step t+{step+1} --")
    for i, var in enumerate(args.targets):
        y_true_step = y_test[:, step, i]
        eval_metrics(y_true_step, base_rolled[:, step, i], name=f"Base t+{step+1} — {var}")
        eval_metrics(y_true_step, multi_preds[:, step, i], name=f"Multi t+{step+1} — {var}")

print("\n-- Overall (flattened across steps) --")
for i, var in enumerate(args.targets):
    eval_metrics(y_test[:, :, i].ravel(), base_rolled[:, :, i].ravel(), name=f"Base t+1..t+6 — {var}")
    eval_metrics(y_test[:, :, i].ravel(), multi_preds[:, :, i].ravel(), name=f"Multi t+1..t+6 — {var}")

print("\n✅ Done. Compared base paper LSTM vs multi-step LSTM using:")
print("   A) t+1 vs t+1, and")
print("   B) Recursive rollout for base LSTM to t+6 vs direct multi-step predictions.")


===== Strategy B: Step-by-step comparison (t+1…t+6) =====

-- Step t+1 --

Base t+1 — rnti_count:
  MSE   : 0.008045
  RMSE  : 0.089696
  MAE   : 0.066063
  R²    : 0.485215
  NRMSE : 0.125351
  MAPE% : 30.246

Multi t+1 — rnti_count:
  MSE   : 0.006707
  RMSE  : 0.081893
  MAE   : 0.062210
  R²    : 0.570880
  NRMSE : 0.114447
  MAPE% : 33.774

Base t+1 — rb_down:
  MSE   : 0.008326
  RMSE  : 0.091249
  MAE   : 0.043579
  R²    : 0.559291
  NRMSE : 0.091397
  MAPE% : 56.740

Multi t+1 — rb_down:
  MSE   : 0.008512
  RMSE  : 0.092262
  MAE   : 0.047042
  R²    : 0.549452
  NRMSE : 0.092412
  MAPE% : 59.624

Base t+1 — rb_up:
  MSE   : 0.012164
  RMSE  : 0.110291
  MAE   : 0.052190
  R²    : 0.610206
  NRMSE : 0.110291
  MAPE% : 149655.780

Multi t+1 — rb_up:
  MSE   : 0.013288
  RMSE  : 0.115274
  MAE   : 0.056397
  R²    : 0.574183
  NRMSE : 0.115274
  MAPE% : 60585.607

Base t+1 — down:
  MSE   : 0.007340
  RMSE  : 0.085676
  MAE   : 0.050102
  R²    : 0.480701
  NRMSE : 0.085995
  

In [ ]:
# import torch, numpy as np, pandas as pd
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ---------- helpers ----------
def _safe_mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    denom = np.clip(np.abs(y_true), eps, None)
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0

def eval_metrics(y_true, y_pred):
    mse  = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    nrmse = rmse / (np.max(y_true) - np.min(y_true) + 1e-8)
    mape = _safe_mape(y_true, y_pred)
    return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2, "NRMSE": nrmse, "MAPE%": mape}

# ---------- STRATEGY A: t+1 vs t+1 → single dataframe ----------
with torch.no_grad():
    xb = torch.tensor(X_test, dtype=torch.float32).to(args.device)
    base_t1  = base_model(xb, device=args.device)            # [N, 5]
    multi_all = multi_model(xb)                               # [N, 6, 5]
base_t1_np  = base_t1.detach().cpu().numpy()                 # [N, 5]
multi_t1_np = multi_all[:, 0, :].detach().cpu().numpy()      # [N, 5]

rows_A = []
for i, var in enumerate(args.targets):
    y_true = y_test[:, 0, i]          # ground-truth t+1
    rows_A.append({"Strategy": "A_t+1", "Model": "Base LSTM (t+1)", "Target": var, **eval_metrics(y_true, base_t1_np[:, i])})
    rows_A.append({"Strategy": "A_t+1", "Model": "Multi-step (t+1)", "Target": var, **eval_metrics(y_true, multi_t1_np[:, i])})

df_strategy_A = pd.DataFrame(rows_A).reset_index(drop=True)

# ---------- STRATEGY B: roll base to t+6 and compare step-by-step → two dataframes ----------
def roll_base_lstm_to_horizon(base_model, X_init, steps, target_pos_in_x, device="cpu"):
    base_model.eval()
    x_win = torch.tensor(X_init, dtype=torch.float32, device=device)  # [N, L, D]
    outs = []
    with torch.no_grad():
        for _ in range(steps):
            y_next = base_model(x_win, device=device)                  # [N, 5]
            outs.append(y_next.unsqueeze(1))
            last_row = x_win[:, -1, :].clone()
            for k, pos in enumerate(target_pos_in_x):
                last_row[:, pos] = y_next[:, k]
            x_win = torch.cat([x_win[:, 1:, :], last_row.unsqueeze(1)], dim=1)
    return torch.cat(outs, dim=1).detach().cpu().numpy()               # [N, steps, 5]

# roll base and get multi-step predictions
base_rolled = roll_base_lstm_to_horizon(
    base_model, X_test, steps=args.forecast_steps, target_pos_in_x=TARGET_POS_IN_X, device=args.device
)  # [N, 6, 5]
multi_preds = multi_all.detach().cpu().numpy()                         # [N, 6, 5]

# Per-step metrics table
rows_B_steps = []
for step in range(args.forecast_steps):
    for i, var in enumerate(args.targets):
        y_true = y_test[:, step, i]
        rows_B_steps.append({
            "Strategy": f"B_t+{step+1}",
            "Step": step+1,
            "Model": "Base LSTM (rolled)",
            "Target": var,
            **eval_metrics(y_true, base_rolled[:, step, i])
        })
        rows_B_steps.append({
            "Strategy": f"B_t+{step+1}",
            "Step": step+1,
            "Model": "Multi-step",
            "Target": var,
            **eval_metrics(y_true, multi_preds[:, step, i])
        })

df_strategy_B_steps = pd.DataFrame(rows_B_steps).reset_index(drop=True)

# Overall (flatten across steps) metrics table
rows_B_overall = []
for i, var in enumerate(args.targets):
    y_true_all   = y_test[:, :, i].ravel()
    base_all     = base_rolled[:, :, i].ravel()
    multi_all_np = multi_preds[:, :, i].ravel()
    rows_B_overall.append({"Strategy": "B_overall", "Model": "Base LSTM (t+1..t+6)", "Target": var, **eval_metrics(y_true_all, base_all)})
    rows_B_overall.append({"Strategy": "B_overall", "Model": "Multi-step (t+1..t+6)", "Target": var, **eval_metrics(y_true_all, multi_all_np)})

df_strategy_B_overall = pd.DataFrame(rows_B_overall).reset_index(drop=True)

# # ---------- Show / Save ----------
# print("\n=== Strategy A: t+1 vs t+1 (both models) ===")
# print(df_strategy_A)

# print("\n=== Strategy B (per step): base rolled vs multi-step ===")
# print(df_strategy_B_steps)

# print("\n=== Strategy B (overall flattened across steps) ===")
# print(df_strategy_B_overall)

# Optional: save to CSV
# df_strategy_A.to_csv("compare_strategy_A_t1.csv", index=False)
# df_strategy_B_steps.to_csv("compare_strategy_B_steps.csv", index=False)
# df_strategy_B_overall.to_csv("compare_strategy_B_overall.csv", index=False)



=== Strategy A: t+1 vs t+1 (both models) ===
  Strategy             Model      Target       MSE      RMSE       MAE  \
0    A_t+1   Base LSTM (t+1)  rnti_count  0.008045  0.089696  0.066063   
1    A_t+1  Multi-step (t+1)  rnti_count  0.006707  0.081893  0.062210   
2    A_t+1   Base LSTM (t+1)     rb_down  0.008326  0.091249  0.043579   
3    A_t+1  Multi-step (t+1)     rb_down  0.008512  0.092262  0.047042   
4    A_t+1   Base LSTM (t+1)       rb_up  0.012164  0.110291  0.052190   
5    A_t+1  Multi-step (t+1)       rb_up  0.013288  0.115274  0.056397   
6    A_t+1   Base LSTM (t+1)        down  0.007340  0.085676  0.050102   
7    A_t+1  Multi-step (t+1)        down  0.007688  0.087681  0.053393   
8    A_t+1   Base LSTM (t+1)          up  0.010708  0.103480  0.045972   
9    A_t+1  Multi-step (t+1)          up  0.012111  0.110049  0.052907   

         R2     NRMSE          MAPE%  
0  0.485215  0.125351      30.246471  
1  0.570880  0.114447      33.773906  
2  0.559291  0.091397 

In [14]:
df_strategy_A

,Strategy,Model,Target,MSE,RMSE,MAE,R2,NRMSE,MAPE%
0,A_t+1,Base LSTM (t+1),rnti_count,0.008045,0.089696,0.066063,0.485215,0.125351,30.246471
1,A_t+1,Multi-step (t+1),rnti_count,0.006707,0.081893,0.062210,0.570880,0.114447,33.773906
2,A_t+1,Base LSTM (t+1),rb_down,0.008326,0.091249,0.043579,0.559291,0.091397,56.740049
3,A_t+1,Multi-step (t+1),rb_down,0.008512,0.092262,0.047042,0.549452,0.092412,59.623999
4,A_t+1,Base LSTM (t+1),rb_up,0.012164,0.110291,0.052190,0.610206,0.110291,149655.779659
5,A_t+1,Multi-step (t+1),rb_up,0.013288,0.115274,0.056397,0.574183,0.115274,60585.606506
6,A_t+1,Base LSTM (t+1),down,0.007340,0.085676,0.050102,0.480701,0.085995,44.473865
7,A_t+1,Multi-step (t+1),down,0.007688,0.087681,0.053393,0.456111,0.088007,50.715272
8,A_t+1,Base LSTM (t+1),up,0.010708,0.103480,0.045972,0.556588,0.103480,181352.646382
9,A_t+1,Multi-step (t+1),up,0.012111,0.110049,0.052907,0.498501,0.110049,108621.611151


In [16]:
df_strategy_B_steps

,Strategy,Step,Model,Target,MSE,RMSE,MAE,R2,NRMSE,MAPE%
0,B_t+1,1,Base LSTM (rolled),rnti_count,0.008045,0.089696,0.066063,0.485215,0.125351,30.246471
1,B_t+1,1,Multi-step,rnti_count,0.006707,0.081893,0.062210,0.570880,0.114447,33.773906
2,B_t+1,1,Base LSTM (rolled),rb_down,0.008326,0.091249,0.043579,0.559291,0.091397,56.740049
3,B_t+1,1,Multi-step,rb_down,0.008512,0.092262,0.047042,0.549452,0.092412,59.623999
4,B_t+1,1,Base LSTM (rolled),rb_up,0.012164,0.110291,0.052190,0.610206,0.110291,149655.779659
5,B_t+1,1,Multi-step,rb_up,0.013288,0.115274,0.056397,0.574183,0.115274,60585.606506
6,B_t+1,1,Base LSTM (rolled),down,0.007340,0.085676,0.050102,0.480701,0.085995,44.473865
7,B_t+1,1,Multi-step,down,0.007688,0.087681,0.053393,0.456111,0.088007,50.715272
8,B_t+1,1,Base LSTM (rolled),up,0.010708,0.103480,0.045972,0.556588,0.103480,181352.646382
9,B_t+1,1,Multi-step,up,0.012111,0.110049,0.052907,0.498501,0.110049,108621.611151


In [15]:
df_strategy_B_overall

,Strategy,Model,Target,MSE,RMSE,MAE,R2,NRMSE,MAPE%
0,B_overall,Base LSTM (t+1..t+6),rnti_count,0.035469,0.188333,0.146515,-1.268102,0.263197,56.931466
1,B_overall,Multi-step (t+1..t+6),rnti_count,0.007651,0.087469,0.067382,0.510763,0.122239,38.331464
2,B_overall,Base LSTM (t+1..t+6),rb_down,0.019102,0.138209,0.069796,-0.011670,0.138433,72.349728
3,B_overall,Multi-step (t+1..t+6),rb_down,0.011023,0.104990,0.056390,0.416203,0.105160,76.147145
4,B_overall,Base LSTM (t+1..t+6),rb_up,0.026200,0.161864,0.089242,0.159088,0.161864,359982.946530
5,B_overall,Multi-step (t+1..t+6),rb_up,0.017975,0.134071,0.067151,0.423073,0.134071,66896.431795
6,B_overall,Base LSTM (t+1..t+6),down,0.017395,0.131889,0.084010,-0.231383,0.132380,56.001357
7,B_overall,Multi-step (t+1..t+6),down,0.009058,0.095175,0.059343,0.358750,0.095530,58.818092
8,B_overall,Base LSTM (t+1..t+6),up,0.020193,0.142103,0.073382,0.161388,0.142103,344137.978736
9,B_overall,Multi-step (t+1..t+6),up,0.015295,0.123672,0.062848,0.364817,0.123672,73101.023204
